# 텍스트 분류 6차 시도

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('final_text_data2.csv', sep='\t', encoding='UTF-8')

In [4]:
df.head(2)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---

In [5]:
df.shape

(10629, 17)

In [6]:
df_O1 = df.loc[df['대상 선정']=='O']
df_O2 = df.loc[df['대상 선정']=='O']
df_X1 = df.loc[df['대상 선정']=='X']
df_X2 = df.loc[df['대상 선정']=='X']
df_O3 = df.loc[df['대상 선정']=='O']
df_O4 = df.loc[df['대상 선정']=='O']
df_X3 = df.loc[df['대상 선정']=='X']
df_X4 = df.loc[df['대상 선정']=='X']
df2 = pd.concat([df_O1,df_X1,df_O2,df_X2,df_O3,df_X3,df_O4,df_X4])
df2.head(2)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
12               W  2017     20171010         ZZ7         B11  조립 문제_미장착/오장착   
14               W  2018     20180725         ZZ7         B11  조립 문제_미장착/오장착   

   trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
12         KV11          CK     J5          AC        91500   
14         KV11          YP     A9          EE        28291   

                                         clm_prtl_sbc 대상 선정       구분   책임구  \
12  [C] 현상-1. 현상  가. 인도후 오디오 작동시 주파수 및 노이즈 발생으로 고객...     O  기능(커넥터)  생산공장   
14  [C] 1. 현상  주행중 엔진경고등 점등2. 점검내용 및 원인  스캔점검결과 P2...     O  기능(커넥터)  생산공장   

     원인                검토결과  
12  미결선  O-기능(커넥터)-생산공장-미결선  
14   기타   O-기능(커넥터)-생산공장-기타

In [7]:
df2.shape

(34128, 17)

In [8]:
df2 = df2.reindex()

# 분류현황

In [9]:
category_count = df2['대상 선정'].value_counts()
category_count

X    22076
O    12052
Name: 대상 선정, dtype: int64

In [10]:
category_count = df2['구분'].value_counts()
category_count

기능(커넥터)    6404
실내부품이음     4200
하체이음       1444
Name: 구분, dtype: int64

In [11]:
category_count = df2['책임구'].value_counts()
category_count

구매      9096
생산공장    2896
연구소       56
Name: 책임구, dtype: int64

In [12]:
category_count = df2['원인'].value_counts()
category_count

부품       9096
조립불량      920
결선불량      900
체결불량      412
기타        400
미결선       148
이종/간섭      84
설계/구조      56
이물         32
Name: 원인, dtype: int64

# 전처리

In [13]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df2['target_pred'] = df2['대상 선정'].astype(str).copy()
# df2 ['charge_pred'] = df2 ['책임구'].astype(str).copy()
# df2 ['cause_pred'] = df2 ['원인'].astype(str).copy()

In [14]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text) # 특수문자 제거
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)   # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)   # 한글, 영문만 남기고 모두 제거하도록 합니다.
    return text

In [15]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [16]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 406 ms


In [17]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 952 ms


In [18]:
df2['clm_prtl_sbc'].head(10)

12          가  인도후 오디오 작동시 주파수 및 노이즈 발생으로 불만 제기   및 ...
14          주행중 엔진경고등 점등   및  스캔점검결과 P       DPF효율저하...
15      테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인시 클립이...
19                     조수석사이드에어백 경고등 점등됨 컨넥터 체결불량으로 재체결
24                        USB 작동 안됨원인 USB 컨넥터 조립 않됨 재조립
25                       어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32        리어LH윈도우작동불량  점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상   트림탈...
38      운전석 뒤도어 유리창 작동 안되어  메인파워스위치 배선인가 점검후뒷도어트림스위치...
40       상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 시...
58           와이퍼 스몰등 후방카메라 오작동 불량   및   리어 루프 배선 R  ...
Name: clm_prtl_sbc, dtype: object

# 학습데이터와 테스트데이터 만들기

In [19]:
df2 = df2.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.

In [20]:
df2.shape

(34128, 18)

In [21]:
df_train = df2.sample(frac=0.8)

In [22]:
df_train.shape

(27302, 18)

In [23]:
df_test = df2.sample(frac=0.2)

In [24]:
df_test.shape

(6826, 18)

# 단어 벡터화

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

stops = ['현상']
vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stops, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 5), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 5), preprocessor=None, stop_words=['현상', '[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [59]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

(27302, 20000)

In [60]:
%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

Wall time: 1.5 s


In [61]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

20000


['abs',
 'abs vdc',
 'abs 모듈',
 'abs모듈',
 'abs모듈교환',
 'abs휠센서',
 'acc',
 'act',
 'acu',
 'acv']

In [62]:
dist = np.sum(train_feature_vector, axis=0)

pd.DataFrame(dist, columns=vocab)

abs  abs vdc  abs 모듈  abs모듈  abs모듈교환  abs휠센서  acc  act  acu  acv  \
0  179       13       7      7       17      11    8   30   10    9   

               ...               히터 회로 제어값  히터 회로 제어값 낮음  히터 회로 제어값 낮음 뱅크  \
0              ...                       8             7               12   

   히터제어  히터컨트롤  히트  힌지  힘이  힘한것으로 확인되어 단품교환후 정상이여서  \
0     8     15  90   7   7                       9   

   힘한것으로 확인되어 단품교환후 정상이여서 고객설명후  
0                             7  

[1 rows x 20000 columns]

# tf-idf 가중치 적용

In [51]:
# tf-idf(Term Frequency-Inverse Document Frequency). 가중치를 구하는 알고리즘.

In [63]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [64]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(27302, 20000)

In [65]:
%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

Wall time: 78 ms


# 머신러닝 : 학습

In [89]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 150, n_jobs = -1, random_state=2000)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=130, n_jobs=-1,
            oob_score=False, random_state=2000, verbose=0,
            warm_start=False)

In [90]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['target_pred']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 8.87 s


# 모델평가

In [91]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 41.4 s


array([0.98297015, 0.98480132, 0.98351648, 0.98076923, 0.98461538])

In [92]:
round(np.mean(score)*100,2)

98.33

# 예측

In [93]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['X', 'O', 'O'], dtype=object)

In [94]:
y_pred.shape

(6826,)

In [95]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(3)

category_pred
0             X
1             O
2             O

In [96]:
df_test['category_pred'] = y_pred

In [97]:
df_test.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
10260               W  2018     20180423         ZZ5         V62   
2921                W  2018     20180326         ZZ3         I11   
7836                W  2017     20171109         ZZ5         Q53   

          ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd  \
10260  브레이크 진동/떨림_제동시         KV11          YP     A9          CC   
2921        작동불량_작동안됨         KV11          YP     A9          AC   
7836     소음/이음_요철로/둔턱         KV11          CK     J5          TT   

      repn_part_cd                                       clm_prtl_sbc 대상 선정  \
10260        51712    프론트 휠 브레이크 디스크 좌 우  변형으로      K 주행중 제동시 핸들 떨...     X   
2921         95400    상기차량 인도후 다음날 아침 스마트키로 도어 언락을 작동하였으나       도어...     X   
7836         81210     요철 주행시 소음 점검 테일 게이트 래치와 스트라이커 유격에 의한 간섭으로 요...     O   

           구분   책임구    원인                검토결과 target_pred category_pred  \
10260     NaN   NaN   NaN                X---           X             X   
2921      NaN   NaN   NaN                X---           X             O   
7836   실내부품이음  생산공장  조립불량  O-실내부품이음-생산공장-조립불량           O             O   

       pred_diff  
10260          1  
2921           0  
7836           1

# 결과검증

In [98]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['대상 선정'] == df_test['category_pred'] ) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
10260               W  2018     20180423         ZZ5         V62   
2921                W  2018     20180326         ZZ3         I11   
7836                W  2017     20171109         ZZ5         Q53   
6392                W  2015     20150325         ZZ5         Q21   
7779                W  2017     20171212         ZZ3         Q53   

            ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd  \
10260    브레이크 진동/떨림_제동시         KV11          YP     A9          CC   
2921          작동불량_작동안됨         KV11          YP     A9          AC   
7836       소음/이음_요철로/둔턱         KV11          CK     J5          TT   
6392   글라스 작동 소음/이음_상승시         KV11          YP     A9          TT   
7779       소음/이음_요철로/둔턱         KV11          CK     J5          TT   

      repn_part_cd                                       clm_prtl_sbc 대상 선정  \
10260        51712    프론트 휠 브레이크 디스크 좌 우  변형으로      K 주행중 제동시 핸들 떨...     X   
2921         95400    상기차량 인도후 다음날 아침 스마트키로 도어 언락을 작동하였으나       도어...     X   
7836         81210     요철 주행시 소음 점검 테일 게이트 래치와 스트라이커 유격에 의한 간섭으로 요...     O   
6392         82530     윈도우 작동시 소음원인  F L H 도어 윈도우 상 하 작동시 런찬넬 변형으로...     O   
7779         82530    주행중에 풍절음 발생점검 주행하면서 청진기를 이용하여 약   킬로 정도 주행시 ...     X   

           구분   책임구    원인                검토결과 target_pred category_pred  \
10260     NaN   NaN   NaN                X---           X             X   
2921      NaN   NaN   NaN                X---           X             O   
7836   실내부품이음  생산공장  조립불량  O-실내부품이음-생산공장-조립불량           O             O   
6392   실내부품이음  생산공장  조립불량  O-실내부품이음-생산공장-조립불량           O             O   
7779      NaN   NaN   NaN                X---           X             X   

       pred_diff  
10260          1  
2921           0  
7836           1  
6392           1  
7779           1

In [99]:
diff_sum = df_test['pred_diff'].sum()
diff_sum

3892

In [100]:
y_pred_len = len(y_pred)
y_pred_len

6826

# 정확도

In [101]:
Acy = round(diff_sum/y_pred_len*100,1)
Acy

57.0